### Testing: Data Cleaning and Quality Control for Pregnancy Deep Phenotyping Metabolomics Data
### Piekos Lab, Kayla Xu
### 01/26/2026

In [1]:
# environment
import pandas as pd 
import numpy as np 


In [2]:
# Metabolomics Files:
### 050725_Sadovsky DP3 Placenta Polar Untargeted_ALL copy.xlsx
### 050725_Sadovsky DP3 Plasma Polar Untargeted_ALL copy.xlsx
placentaPos_raw = pd.read_excel('/Users/kaylaxu/Desktop/data/raw_data/050725_Sadovsky DP3 Placenta Polar Untargeted_ALL copy.xlsx', sheet_name="POS Compounds",skiprows=2)
plasmaPos_raw = pd.read_excel('/Users/kaylaxu/Desktop/data/raw_data/050725_Sadovsky DP3 Plasma Polar Untargeted_ALL copy.xlsx', sheet_name="POS Compounds",skiprows=2)

In [6]:
# extract "Area" information
placentaPos = placentaPos_raw
not_unnamed =["Unnamed" not in s for s in placentaPos.columns]
sample_area = placentaPos.iloc[0:1, not_unnamed]
sample_area = sample_area.set_index("Sample ID").transpose()
sample_area.columns = ["Area"]
sample_area.to_csv("/Users/kaylaxu/Desktop/data/clean_data/MTBL_placenta/pos_sample_area.csv")

In [7]:
# Extract compound metadata
not_sample =["DP3" not in s and "Pooled" not in s for s in placentaPos.columns]
placentaPos_comps = placentaPos.iloc[:, not_sample]
placentaPos_comps = placentaPos_comps.rename(columns=placentaPos_comps.iloc[0,:]).drop([0])
placentaPos_comps.iloc[0,0] = "p01" # D3-alanine-ISTD
placentaPos_comps.iloc[1,0] = "p02" # 13C1-creatinine_ISTD
placentaPos_comps.to_csv("/Users/kaylaxu/Desktop/data/clean_data/MTBL_placenta/pos_compounds.csv")


In [9]:
# Extract expression data
is_expression = ["DP3" in s or "Pooled" in s for s in placentaPos.columns]
expression = placentaPos.iloc[:, is_expression]
expression = expression.drop([0])
expression.index = placentaPos_comps["Export Order"]
expression = expression.transpose()
expression.to_csv("/Users/kaylaxu/Desktop/data/clean_data/MTBL_placenta/pos_expression.csv")

In [12]:
expression.index

Index(['Pooled Cntrl', 'Pooled Cntrl.1', 'Pooled Cntrl.2', 'Pooled Cntrl.3',
       'Pooled Cntrl.4', 'Pooled Cntrl.5', 'Pooled Cntrl.6', 'Pooled Cntrl.7',
       'Pooled Cntrl.8', 'Pooled Cntrl.9',
       ...
       'DP3-0387', 'DP3-0389', 'DP3-0398E', 'DP3-0399', 'DP3-0404', 'DP3-0409',
       'DP3-0416', 'DP3-0419', 'DP3-0420', 'DP3-0423E'],
      dtype='str', length=137)

In [3]:
def get_area(df):
    is_sample = df.columns.notna()
    temp = df.iloc[0:1,is_sample]
    temp = temp.set_index("Sample ID").transpose()
    temp.columns = ["Area"]
    return temp




In [4]:
# extract compound metadata
def get_compounds(df):
    not_sample = df.columns.isna()
    temp = df.iloc[:, not_sample]
    temp.columns = temp.iloc[0,:]
    temp = temp.drop(temp.index[0])
    temp.iloc[0,0] = "01" # two moleclues not in the export order
    temp.iloc[1,0] = "02" # slightly different for each file
    return temp


In [18]:

# extract expression data
def get_expression(df, ids):
    is_sample = df.columns.notna()
    temp = df.iloc[:, is_sample].drop(columns="Sample ID").drop(df.index[0])
    print(temp)
    temp.index = ids
    temp = temp.transpose()
    return temp



In [6]:
# helper function
def extract_data(file_input, file_output):
    #file = open(file_input, mode="r")
    file = file_input
    file_pos = pd.read_excel(file, sheet_name="POS Compounds",header=None).dropna(how='all')
    file_neg = pd.read_excel(file, sheet_name="NEG Compounds",header=None).dropna(how='all')
    file_pos.columns = file_pos.iloc[0,:]
    file_neg.columns = file_neg.iloc[0,:]
    file_pos = file_pos.drop(file_pos.index[0])
    file_neg = file_neg.drop(file_neg.index[0])

    #extract positive compound files
    get_area(file_pos).to_csv(file_output + "/pos_sampleArea.csv")
    pos_comp = get_compounds(file_pos)
    pos_comp.to_csv(file_output + "/pos_compounds.csv")
    get_expression(file_pos, pos_comp.iloc[:,0]).to_csv(file_output + "/pos_expression.csv")

    #extract negative compound files
    get_area(file_neg).to_csv(file_output + "/neg_sampleArea.csv")
    neg_comp = get_compounds(file_neg)
    neg_comp.to_csv(file_output + "/neg_compounds.csv")
    get_expression(file_neg, neg_comp.iloc[:,0]).to_csv(file_output + "/neg_expression.csv")

In [19]:
extract_data('/Users/kaylaxu/Desktop/data/raw_data/050725_Sadovsky DP3 Placenta Polar Untargeted_ALL copy.xlsx', '/Users/kaylaxu/Desktop/data/clean_data/MTBL_placenta/test')

2         Pooled Cntrl      Pooled Cntrl      Pooled Cntrl      Pooled Cntrl  \
4                  NaN               NaN               NaN               NaN   
5                  NaN               NaN               NaN               NaN   
6     1529697732.10807  1368644592.09096  1529095145.48298  1594294947.22906   
7     960934888.044903  1017308011.36853  1024927927.19107  1061045171.20838   
8     696936755.396744  729792167.056774  753789577.578159  772700856.622067   
...                ...               ...               ...               ...   
3705      39079.646816       37595.24349      41865.668458      27640.514842   
3706      32186.048274       14915.13476      30473.225647      34969.834441   
3707       1601.922548       1483.575237       1565.337035       2063.441685   
3708       4823.045917       4975.115785      10850.491229       3151.114966   
3709       2013.510488       1822.346712        1234.04052       1291.215513   

2         Pooled Cntrl      Pooled Cntr